In [555]:
with open('19a.txt') as f:
    lines = f.readlines()

lines = [line.strip() for line in lines]

In [556]:
from typing import Tuple
from math import copysign
import numpy as np

In [557]:
def rotation_sign(s: str) -> int:
    if (s[0]) == '-': return -1
    return 1

In [558]:
def rotate_transform(b_pos: Tuple[int, int, int], orientation: str) -> Tuple[int, int, int]:
    orientation1 = orientation.split(',')
    if orientation1[0] == 'x' or orientation1[0] == '-x': xi1 = 0
    elif orientation1[0] == 'y' or orientation1[0] == '-y': yi1 = 0
    elif orientation1[0] == 'z' or orientation1[0] =='-z': zi1 = 0
    else: print("o0")
    
    if orientation1[1] == 'x' or orientation1[1] == '-x': xi1 = 1
    elif orientation1[1] == 'y' or orientation1[1] == '-y': yi1 = 1
    elif orientation1[1] == 'z' or orientation1[1] == '-z': zi1 = 1
    else: print('o1')

    if orientation1[2] == 'x' or orientation1[2] == '-x': xi1 = 2
    elif orientation1[2] == 'y' or orientation1[2] == '-y': yi1 = 2
    elif orientation1[2] == 'z' or orientation1[2] == '-z': zi1 = 2
    else: print('o2')

    x1sgn = -1 if orientation1[xi1][0] == '-' else 1
    y1sgn = -1 if orientation1[yi1][0] == '-' else 1
    z1sgn = -1 if orientation1[zi1][0] == '-' else 1

    return (b_pos[xi1]*x1sgn, b_pos[yi1]*y1sgn, b_pos[zi1]*z1sgn)
    

In [559]:
def same_beacons(a1: Tuple[int, int, int], a2: Tuple[int, int, int], b1: Tuple[int, int, int], b2: Tuple[int, int, int], rotation: str) -> bool: 
        i_coord, j_coord, k_coord = rotation.split(',')
        x_diff = a1[0] - a2[0]
        y_diff = a1[1] - a2[1]
        z_diff = a1[2] - a2[2]

        i_diff = (b1[0] - b2[0]) * rotation_sign(i_coord)
        j_diff = (b1[1] - b2[1]) * rotation_sign(j_coord)
        k_diff = (b1[2] - b2[2]) * rotation_sign(k_coord)

        if i_coord[-1] == 'x': x_comp = x_diff == i_diff
        elif i_coord[-1] == 'y': x_comp = y_diff == i_diff
        elif i_coord[-1] == 'z': x_comp = z_diff == i_diff
        else: print('i-coord error.')

        if j_coord[-1] == 'x': y_comp = x_diff == j_diff
        elif j_coord[-1] == 'y': y_comp = y_diff == j_diff
        elif j_coord[-1] == 'z': y_comp = z_diff == j_diff
        else: print('j-coord error.')

        if k_coord[-1] == 'x': z_comp = x_diff == k_diff
        elif k_coord[-1] == 'y': z_comp = y_diff == k_diff
        elif k_coord[-1] == 'z': z_comp = z_diff == k_diff
        else: print('k-coord error.')

        return x_comp and y_comp and z_comp

In [560]:
def fast_same(a1,a2,b1,b2) -> bool:
    aa1 = np.array(a1)
    aa2 = np.array(a2)
    bb1 = np.array(b1)
    bb2 = np.array(b2)
    dista = np.linalg.norm(aa1-aa2)
    distb = np.linalg.norm(bb1-bb2)
    return dista == distb


In [562]:
class Scanner:

    def __init__(self, list_of_beacons):
        self.list_of_beacons = list_of_beacons
        self.shared_beacons = {}
        self.zero_pos = (0,0,0)
        with open('19_combinations.txt') as f:
            self.rotations = f.readlines()
        self.rotations = [r.strip() for r in self.rotations]
        self.orientation = 'x,y,z'
        self.set = False
        self.potential_hits = {}
        
    
    def add_beacon(self, beacon):
        self.list_of_beacons.append(beacon)

    def fast_overlap(self, other) -> bool:
        common_beacons = []
        other_common_beacons = []
        for i in range(len(self.list_of_beacons)):
            if len(common_beacons) >= 12: break
            for j in range(i+1,len(self.list_of_beacons)):
                if len(common_beacons) >= 12: break
                for k in range(len(other.list_of_beacons)):
                    if len(common_beacons) >= 12: break
                    for l in range(k+1, len(other.list_of_beacons)):
                        # same: whether or not these two pairs of beacons have the same magnitude of difference.
                        same = fast_same(self.list_of_beacons[i],self.list_of_beacons[j],other.list_of_beacons[k],other.list_of_beacons[l])
                        if same:
                            if i not in common_beacons: common_beacons.append(i)
                            if j not in common_beacons: common_beacons.append(j)
                            if k not in other_common_beacons: other_common_beacons.append(k)
                            if l not in other_common_beacons: other_common_beacons.append(l)
                            break
        if len(common_beacons) >= 12:
            self.potential_hits[other] = common_beacons
            other.potential_hits[self] = other_common_beacons
            print('yay')
        return len(common_beacons) >= 12
           

    def check_overlap(self, other) -> bool:
        common_beacons = []
        other_common_beacons = []
        good_rotation = 'x,y,z'
        for rotation in self.rotations:
            common_beacons = []
            other_common_beacons = []
            for i in self.potential_hits[other]:
                if len(common_beacons) >= 12: break
                for j in self.potential_hits[other]:
                    if i!=j:
                        if len(common_beacons) >= 12: break
                        for k in other.potential_hits[self]:
                            if len(common_beacons) >= 12: break
                            for l in other.potential_hits[self]:
                                if k!=l:
                                    # same: whether or not these two pairs of beacons have the same magnitude of difference.
                                    same = same_beacons(self.list_of_beacons[i], self.list_of_beacons[j], other.list_of_beacons[k], other.list_of_beacons[l], rotation)
                                    if same:
                                        if i not in common_beacons: common_beacons.append(i)
                                        if j not in common_beacons: common_beacons.append(j)
                                        if k not in other_common_beacons: other_common_beacons.append(k)
                                        if l not in other_common_beacons: other_common_beacons.append(l)
                                        break
            if len(common_beacons) >= 12:
                # figure out the absolute rotation right now ??
                # '-y x -z' base
                # 'x z -y' curr
                # what is the absolute rotation?
                # [0] value -> -y
                # [1] value -> -z
                # [2] value -> -x
                print(rotation)
                super_rotation = self.orientation
                s0, s1, s2 = super_rotation.split(',')
                zero,one,two = rotation.split(',')

                if zero == 'x': good_x = s0
                elif zero == '-x':
                    if s0[0] == '-': good_x = s0[-1]
                    else: good_x = '-'+s0
                elif zero == 'y': good_x = s1
                elif zero == '-y':
                    if s1[0] == '-': good_x = s1[-1]
                    else: good_x = '-'+s1
                elif zero == 'z': good_x = s2
                elif zero == '-z':
                    if s2[0] == '-': good_x = s2[-1]
                    else: good_x = '-'+s2

                if one == 'x': good_y = s0
                elif one == '-x':
                    if s0[0] == '-': good_y = s0[-1]
                    else: good_y = '-'+s0
                elif one == 'y': good_y = s1
                elif one == '-y':
                    if s1[0] == '-': good_y = s1[-1]
                    else: good_y = '-'+s1
                elif one == 'z': good_y = s2
                elif one == '-z':
                    if s2[0] == '-': good_y = s2[-1]
                    else: good_y = '-'+s2
                
                if two == 'x': good_z = s0
                elif two == '-x':
                    if s0[0] == '-': good_z = s0[-1]
                    else: good_z = '-'+s0
                elif two == 'y': good_z = s1
                elif two == '-y':
                    if s1[0] == '-': good_z = s1[-1]
                    else: good_z = '-'+s1
                elif two == 'z': good_z = s2
                elif two == '-z':
                    if s2[0] == '-': good_z = s2[-1]
                    else: good_z = '-'+s2

                good_rotation = good_x + ',' + good_y + ',' + good_z
                # print(good_rotation)
                break
        if len(common_beacons) >= 12:
            self.shared_beacons[other] = common_beacons
            other.shared_beacons[self] = other_common_beacons
            other.orientation = good_rotation
        return len(common_beacons) >= 12

    def set_zero_pos(self,coords):
        self.zero_pos = (coords[0],coords[1],coords[2])
        self.set = True
        

In [564]:
scanners = []
for line in lines:
    if len(line) > 0 and line[1] == '-':
        curr_scanner = Scanner([])
        scanners.append(curr_scanner)
    elif len(line) > 1:
        coords = line.split(',')
        curr_scanner.add_beacon((int(coords[0]), int(coords[1]), int(coords[2])))
        # print(curr_scanner.list_of_beacons)

In [565]:
def same_sign(num1,num2):
    return copysign(1,num1) == copysign(1,num2)

In [566]:
def find_scanner_pos(scanner1: Scanner, scanner2: Scanner) -> Tuple[int, int, int]:
    # we either add or subtract each of the coords.
    print('orientation 1: ' + scanner1.orientation)
    print('orientation 2: ' + scanner2.orientation)
    orientation1 = scanner1.orientation.split(',')
    absolute_orientation1 = (orientation1[0][-1], orientation1[1][-1], orientation1[2][-1])
    if orientation1[0] == 'x' or orientation1[0] == '-x': xi1 = 0
    elif orientation1[0] == 'y' or orientation1[0] == '-y': yi1 = 0
    elif orientation1[0] == 'z' or orientation1[0] =='-z': zi1 = 0
    else: print("o0")
    
    if orientation1[1] == 'x' or orientation1[1] == '-x': xi1 = 1
    elif orientation1[1] == 'y' or orientation1[1] == '-y': yi1 = 1
    elif orientation1[1] == 'z' or orientation1[1] == '-z': zi1 = 1
    else: print('o1')

    if orientation1[2] == 'x' or orientation1[2] == '-x': xi1 = 2
    elif orientation1[2] == 'y' or orientation1[2] == '-y': yi1 = 2
    elif orientation1[2] == 'z' or orientation1[2] == '-z': zi1 = 2
    else: print('o2')

    x1sgn = -1 if orientation1[xi1][0] == '-' else 1
    y1sgn = -1 if orientation1[yi1][0] == '-' else 1
    z1sgn = -1 if orientation1[zi1][0] == '-' else 1

    orientation2 = scanner2.orientation.split(',')
    # absolute_orientation2 = (orientation2[0][-1], orientation2[1][-1], orientation2[2][-1])
    if orientation2[0] == 'x' or orientation2[0] == '-x': xi2 = 0
    elif orientation2[0] == 'y' or orientation2[0] == '-y': yi2 = 0
    elif orientation2[0] == 'z' or orientation2[0] =='-z': zi2 = 0
    
    if orientation2[1] == 'x' or orientation2[1] == '-x': xi2 = 1
    elif orientation2[1] == 'y' or orientation2[1] == '-y': yi2 = 1
    elif orientation2[1] == 'z' or orientation2[1] == '-z': zi2 = 1
    else: print('o1')

    if orientation2[2] == 'x' or orientation2[2] == '-x': xi2 = 2
    elif orientation2[2] == 'y' or orientation2[2] == '-y': yi2 = 2
    elif orientation2[2] == 'z' or orientation2[2] == '-z': zi2 = 2
    else: print('o2')
    # if absolute_orientation2[0] == absolute_orientation1[xi1]: xi2 = 0
    # elif absolute_orientation2[0] == absolute_orientation1[yi1]: yi2 = 0
    # elif absolute_orientation2[0] == absolute_orientation1[zi1]: zi2 = 0
    
    # if absolute_orientation2[1] == absolute_orientation1[xi1]: xi2 = 1
    # elif absolute_orientation2[1] == absolute_orientation1[yi1]: yi2 = 1
    # elif absolute_orientation2[1] == absolute_orientation1[zi1]: zi2 = 1

    # if absolute_orientation2[2] == absolute_orientation1[xi1]: xi2 = 2
    # elif absolute_orientation2[2] == absolute_orientation1[yi1]: yi2 = 2
    # elif absolute_orientation2[2] == absolute_orientation1[zi1]: zi2 = 2

    x2sgn = -1 if orientation2[xi2][0] == '-' else 1
    y2sgn = -1 if orientation2[yi2][0] == '-' else 1
    z2sgn = -1 if orientation2[zi2][0] == '-' else 1

    beacon_ls1 = scanner1.list_of_beacons
    beacon_ls2 = scanner2.list_of_beacons
    shared_indices1 = scanner1.shared_beacons[scanner2]
    shared_indices2 = scanner2.shared_beacons[scanner1]
    shared_coords1 = [beacon_ls1[x] for x in shared_indices1]
    shared_coords2 = [beacon_ls2[x] for x in shared_indices2]

    x = shared_coords1[0][xi1] * x1sgn - shared_coords2[0][xi2] * x2sgn  + scanner1.zero_pos[0]
    y = shared_coords1[0][yi1] * y1sgn - shared_coords2[0][yi2] * y2sgn + scanner1.zero_pos[1]
    z = shared_coords1[0][zi1] * z1sgn - shared_coords2[0][zi2] * z2sgn  + scanner1.zero_pos[2] 
    
    return (x,y,z)

In [571]:
scnr0 = scanners[0]
scnr0.set = True
scnr1 = scanners[1]
base_scnr = scnr0
for idx in range(len(scanners)):
    base_scnr = scanners[idx]
    if base_scnr.set:
        for i in range(len(scanners)):
            if i != idx:
                curr_scnr = scanners[i]
                if not curr_scnr.set and base_scnr.fast_overlap(curr_scnr) and base_scnr.check_overlap(curr_scnr):
                    curr_scnr.set_zero_pos(find_scanner_pos(base_scnr, curr_scnr))
                    print(base_scnr.zero_pos)
                    # print("or1: " + base_scnr.orientation)
                    # print("or2: " + curr_scnr.orientation)
                    print("from scanner #" + str(idx))
                    print("scanner #" + str(i) + "\t" + str(curr_scnr.zero_pos))

yay
-z,x,-y
orientation 1: x,y,z
orientation 2: -z,x,-y
(0, 0, 0)
from scanner #0
scanner #7	(35, 41, 1043)
yay
z,x,y
orientation 1: x,y,z
orientation 2: z,x,y
(0, 0, 0)
from scanner #0
scanner #9	(1254, 75, -39)
yay
-y,-x,-z
orientation 1: -z,x,-y
orientation 2: -x,z,y
(35, 41, 1043)
from scanner #7
scanner #2	(3, 122, 2402)
yay
z,x,y
orientation 1: -z,x,-y
orientation 2: -y,-z,x
(35, 41, 1043)
from scanner #7
scanner #17	(1075, 30, 1147)
yay
z,-y,x
orientation 1: z,x,y
orientation 2: y,-x,z
(1254, 75, -39)
from scanner #9
scanner #21	(1157, -1099, -185)
yay
y,x,-z
orientation 1: -y,-z,x
orientation 2: -z,-y,-x
(1075, 30, 1147)
from scanner #17
scanner #14	(1073, 166, 2276)
yay
x,-y,-z
orientation 1: -y,-z,x
orientation 2: -y,z,-x
(1075, 30, 1147)
from scanner #17
scanner #15	(1103, 1340, 1046)
yay
y,x,-z
orientation 1: -y,-z,x
orientation 2: -z,-y,-x
(1075, 30, 1147)
from scanner #17
scanner #25	(1160, -1041, 1026)
yay
y,-x,z
orientation 1: y,-x,z
orientation 2: -x,-y,z
(1157, -1099,

In [576]:
counter = 0
while counter < 26:
    counter = 0
    base_scnr = scnr0
    for idx in range(len(scanners)):
        base_scnr = scanners[idx]
        if base_scnr.set:
            for i in range(len(scanners)):
                if i != idx:
                    curr_scnr = scanners[i]
                    if not curr_scnr.set and base_scnr.fast_overlap(curr_scnr) and base_scnr.check_overlap(curr_scnr):
                        curr_scnr.set_zero_pos(find_scanner_pos(base_scnr, curr_scnr))
                        print(base_scnr.zero_pos)
                        # print("or1: " + base_scnr.orientation)
                        # print("or2: " + curr_scnr.orientation)
                        print("from scanner #" + str(idx))
                        print("scanner #" + str(i) + "\t" + str(curr_scnr.zero_pos))
    for s in scanners:
        if s.set: counter += 1
    print(counter)


yay
x,z,-y
orientation 1: -x,z,y
orientation 2: -x,y,-z
(3, 122, 2402)
from scanner #2
scanner #1	(30, 58, 3463)
yay
-y,z,-x
orientation 1: -x,z,y
orientation 2: -z,y,x
(3, 122, 2402)
from scanner #2
scanner #16	(34, -1160, 2248)
yay
x,-z,y
orientation 1: z,-y,x
orientation 2: z,-x,-y
(1175, -2333, 1161)
from scanner #4
scanner #18	(1121, -3548, 1143)
yay
-z,-x,y
orientation 1: z,-y,x
orientation 2: -x,-z,-y
(1175, -2333, 1161)
from scanner #4
scanner #19	(1082, -2324, 2251)
yay
-y,-z,x
orientation 1: -z,-y,-x
orientation 2: y,x,-z
(1073, 166, 2276)
from scanner #14
scanner #11	(2451, 155, 2221)
yay
-x,y,-z
orientation 1: -y,z,-x
orientation 2: y,z,x
(1103, 1340, 1046)
from scanner #15
scanner #12	(1103, 2383, 1149)
yay
-y,-z,x
orientation 1: -z,y,x
orientation 2: -y,-x,-z
(34, -1160, 2248)
from scanner #16
scanner #20	(-72, -2250, 2305)
yay
-y,z,-x
orientation 1: z,-x,-y
orientation 2: x,-y,-z
(1121, -3548, 1143)
from scanner #18
scanner #6	(58, -3440, 1180)
yay
y,x,-z
orientation 1: 

### Part 1

In [577]:
beacons_set = set()
for scanner in scanners:
    for beac in scanner.list_of_beacons:
        b = rotate_transform(beac, scanner.orientation)
        absolute_b0 = int(b[0]) + int(scanner.zero_pos[0])
        absolute_b1 = int(b[1]) + int(scanner.zero_pos[1])
        absolute_b2 = int(b[2]) + int(scanner.zero_pos[2])
        beacons_set.add((absolute_b0,absolute_b1,absolute_b2))
        

print(len(beacons_set))
beacons_set
## 458 too high.!

332


{(-1932, -4021, 1736),
 (-1869, -3989, 1573),
 (-1863, -4286, 297),
 (-1853, -3973, 1625),
 (-1836, -2733, 627),
 (-1746, -4369, 238),
 (-1739, -2715, 469),
 (-1734, -3051, 1767),
 (-1713, -2824, 627),
 (-1685, -3020, 1689),
 (-1672, -4330, 332),
 (-1539, -3043, 1684),
 (-1336, -3429, 1203),
 (-1269, -3597, 1075),
 (-891, 910, 577),
 (-853, -4241, 2776),
 (-825, -457, 5179),
 (-805, -621, 725),
 (-805, 941, 592),
 (-794, -601, 2710),
 (-791, -4240, 2667),
 (-791, 423, 3947),
 (-778, -4182, 2633),
 (-776, -509, 4004),
 (-771, 970, 1782),
 (-764, 969, 465),
 (-761, 604, 2782),
 (-754, -390, 5117),
 (-753, 795, 2804),
 (-751, -430, 5231),
 (-744, 385, 4044),
 (-730, 602, -747),
 (-730, 838, 1901),
 (-727, 486, 3998),
 (-722, -633, 630),
 (-714, -1867, 5467),
 (-707, -4059, 1585),
 (-706, -5426, 1860),
 (-705, -430, 4027),
 (-703, -3989, 1804),
 (-703, 752, 2859),
 (-702, -5522, 1932),
 (-698, -1505, 1775),
 (-695, -589, 2706),
 (-691, -1565, 1762),
 (-686, -663, 2636),
 (-678, 939, 1948),

### Part 2

In [579]:
def manhattan_dist(a: Tuple[int, int, int], b: Tuple[int, int, int]) -> int:
    return abs(a[0]-b[0]) + abs(a[1]-b[1]) + abs(a[2]-b[2])

In [580]:
max_dist = 0
for i in range(len(scanners)):
    for j in range(len(scanners)):
        pos1 = scanners[i].zero_pos
        pos2 = scanners[j].zero_pos
        dist = manhattan_dist(pos1,pos2)
        if dist > max_dist: 
            max_dist = dist
            print(max_dist)
print("FINAL ANSWER: " + str(max_dist))

3551
5905
5942
6013
6890
7231
7273
7327
8242
8288
8325
8372
8415
8505
8507
FINAL ANSWER: 8507
